## Le modèle de Malthus {#sec-malthus}

Nous considérons le modèle proposé par @malthus :
$$
\dot x = (n-m)x,
$$ {#eq-malthus}
avec $n$ le taux de natalité, et $m$ le taux de mortalité.

### Préliminaires

Pour simuler ce modèle, c'est à dire intégrer numériquement les solutions au problème de Cauchy correspondant à l'@eq-malthus avec $x(0)=x_0\geq0$, nous allons utiliser les routines de `DifferentialEquations.jl` avec la méthode par défaut. Nous utiliserons aussi le package `Plots.jl` pour les représentations graphiques^[si vous n'avez jamais installé un package, il faudra commencer par le faire avant de pouvoir l'utiliser...].


In [ ]:
using Plots, DifferentialEquations

### Conditions initiales, paramètres et temps

Nous définissions la condition initiale de la simulation :


In [ ]:
x0 = 0.1

Nous définissons les paramètres, et les encapsulons dans un vecteur de paramètres :


In [ ]:
n = 3.0     # natalité
m = 2.0     # morrtalité
par_malthus = [n, m] # packing

Enfin, les propriétés du temps d'intégration :

In [ ]:
tspan = (0.0, 10.0)
tstep = 0.1

### Système dynamique

On définit le modèle comme une fonction renvoyant la dérivée de $x$ (comme `SVector`) en fonction de l'état, de paramètres, et du temps. L'ordre des arguments est important, c'est sous cette forme que les routines d'intégration attendent le modèle.


In [ ]:
function malthus(u, p, t)
    n, m = p        # unpacking
    x = u[1]        # use x notation
    dx = (n-m)x     # return derivative
end

### Intégration

On commence par définir le problème de Cauchy à intégrer, comme un `ODEProblem` avec arguments: le modèle, la condition initiale, les bornes d'intégration, les paramètres ainsi que l'option `saveat = tstep` permettant de récupérer la solution tout les `tstep` pas de temps (il y a plein d'options de sortie de l'intégration, dont même une fonction du temps !)


In [ ]:
prob_malthus = ODEProblem(malthus,          # modèle
                          x0,               # condition initiale
                          tspan,            # tspan
                          par_malthus,      # paramètres
                          saveat = tstep)   # option de sortie

On intègre le modèle via `solve`, défini par `DifferentialEquations.jl` pour des `struct` de type `ODEProblem` :


In [ ]:
sol_malthus = solve(prob_malthus)

Le type de solution renvoyée par le solveur est assez complexe et comprend de nombreux champs informatifs sur le calcul. On peut accéder au temps de simulation via `sol_malthus.t` :


In [ ]:
#| output: true
first(sol_malthus.t, 3)

ainsi qu'aux valeurs de la variable $x$ calculées le long du temps via `sol_matlhus.u` :


In [ ]:
#| output: true
first(sol_malthus.u, 3)

Même si ce n'est pas indispensable, il est possible de transformer la solution renvoyée facilement en `DataFrame`, qui peut permettre des manipulations plus faciles.


In [ ]:
using DataFrames

sol_malthus = DataFrame(sol_malthus)
rename!(sol_malthus, :timestamp => :time, :value => :x)

si bien que :


In [ ]:
#| output: true
first(sol_malthus, 3)

### Représentation graphique


On peut représenter graphiquement la simulation de la croissance de la population au cours du temps (ici via le dataframe).


In [ ]:
#| output: true
plot(sol_malthus.time, sol_malthus.x,
    palette = :tab10,
    linewidth = 2,
    title = "\n Modèle de Malthus \$n=$n, m=$m\$",
    label = "population \$x\$",
    ylabel = "densité de population \$x(t)\$",
    xlabel = "temps \$t\$",
    margin = .5Plots.cm,
    topmargin = 1Plots.cm)

## Le modèle logistique {#sec-logistic}

Nous considérons ici le modèle "logistique" proposé par @verhulst1838 :

$$
\dot x = r x \left(1-\frac{x}{K}\right),
$$ {#eq-logistic}
avec $r$ le taux de croissance intrinsèque de la population et $K$ la capacité de charge de l'environnement.


Il n'y a pas de difficulté particulière par rapport aux simulations précedentes. On va réutiliser la condition initiale et les paramètres de temps définis précédemment.

Il faut cependant définir les paramètres du modèle @eq-logistic :

In [ ]:
#| code-fold: true
r = 1.0      # natalité
K = 10.0     # morrtalité
par_logistic = [r, K] # packing

ainsi que le nouveau système dynamique :

In [ ]:
#| code-fold: true
function logistic(u, p, t)
    r, K = p            # unpacking
    x = u[1]            # use x notation
    dx = r*x*(1-x/K)    # return derivative
end

et le problème de Cauchy correspondant :

In [ ]:
#| code-fold: true
prob_logistic = ODEProblem(logistic,        # modèle
                          x0,               # condition initiale
                          tspan,            # tspan
                          par_logistic,     # paramètres
                          saveat = tstep)   # option de sortie

On simule et on transforme la solution en dataframe :

In [ ]:
#| code-fold: true
sol_logistic = solve(prob_logistic)

sol_logistic = DataFrame(sol_logistic)
rename!(sol_logistic, :timestamp => :time, :value => :x)

On trace la solution, en rajoutant les équilibres stable ($x=K$) et instable ($x=0$) :

In [ ]:
#| code-fold: true
#| output: true
# solution
plot(sol_logistic.time, sol_logistic.x,
    palette = :tab10,
    linewidth = 2,
    title = "\n Modèle logistique \$r=$r, K=$K\$",
    label = "population \$x\$",
    legend = :right,
    ylabel = "densité de population \$x(t)\$",
    xlabel = "temps \$t\$",
    margin = .5Plots.cm,
    topmargin = 1Plots.cm)

# équilibre 0
plot!(sol_logistic.time, zeros(length(sol_logistic.time)),
      color = "red",
      linewidth = 2,
      linestyle = :dash,
      linealpha = .5,
      label = "équilibre instable")

# équilibre 1
plot!(sol_logistic.time, ones(length(sol_logistic.time)).*K,
      color = "green",
      linewidth = 2,
      linestyle = :dash,
      linealpha = .5,
      label = "équilibre stable")

\


## Effets Allee {#sec-allee}

On s'intéresse à un modèle de dynamique de population avec "effets Allee forts", souvent attribué à @gruntfest1997^[mais de nombreuses variations de cette forme polynomiale existent dans la littérature depuis @bazykin1985] :

$$
\dot x = r x \left(\frac{x}{\epsilon}-1\right)\left(1-\frac{x}{K}\right),
$$ {#eq-allee}
avec $r$ le taux de croissance intrinsèque de la population (par analogie avec la logistique), $K$ la capacité de charge de l'environnement et $\epsilon$ le seuil en dessous duquel la population n'est pas viable ('seuil de Allee').

Nous souhaitons illustrer la bistabilité caractéristique du comportement de ce modèle à partir de la simulation depuis plusieurs conditions initiales.

Commençons par définir le modèle et les paramètres (en conservant les $r$ et $K$ précédents):

In [ ]:
#| code-fold: true
epsilon = 2                 # seuil de Allee
par_allee = [r, K, epsilon] # packing
tspan_allee = (0.0, 3.0)

function allee(u, p, t)
    r, K, epsilon = p        # unpacking
    x = u[1]                 # use x notation
    dx = r*x*(x/epsilon - 1)*(1 - x/K)     # return derivative
end

Nous définissons une fonction qui réalise une simulation en un seul appel :


In [ ]:
function int_allee(x0, tspan = tspan_allee, param = par_allee)
    prob_allee = ODEProblem(allee,          # modèle
                          x0,               # condition initiale
                          tspan,            # tspan
                          param,            # paramètres
                          saveat = tstep)   # option de sortie

    sol_allee = solve(prob_allee)
    sol_allee = DataFrame(sol_allee)
    rename!(sol_allee, :timestamp => :time, :value => :x)
end

Nous définissons un vecteur de conditions initiales différentes :

In [ ]:
x0step = 1.35
x0vec = x0step:x0step:K

Finalement on réalise la figure, en initiant un graphique `P` et en faisant une boucle pour tracer chacune des simulations correspondant aux différentes conditions initiales :

In [ ]:
#| output: true
# custom color palette
init_cgrad = palette([:steelblue, :lightblue], length(x0vec))

# initialisation du graphique
P = plot(palette = init_cgrad,
         title = "\n Modèle à effets Allee \$r=$r, K=$K\$, \$ϵ=$epsilon\$",
         legend = :right,
         ylabel = "densité de population \$x(t)\$",
         xlabel = "temps \$t\$",
         margin = .5Plots.cm,
         topmargin = 1Plots.cm)

# boucle de plot avec intégration pour differentes conditions initiales
for x0 in x0vec
    plot!(P, int_allee(x0).time, int_allee(x0).x,
          linewidth = 2,
          label = "")
end

# équilibres
plot!(P, int_allee(0).time, zeros(length(int_allee(0).time)),
      color = "green",
      linewidth = 2,
      linestyle = :dash,
      linealpha = .5,
      label = "équilibre stable")
plot!(P, int_allee(0).time, ones(length(int_allee(0).time)).*epsilon,
      color = "red",
      linewidth = 2,
      linestyle = :dash,
      linealpha = .5,
      label = "équilibre instable")
plot!(P, int_allee(0).time, ones(length(int_allee(0).time)).*K,
      color = "green",
      linewidth = 2,
      linestyle = :dash,
      linealpha = .5,
      label = "")

display(P)      # actually shows the plot P

\

Passons aux [populations exploitées](pop_exploitees.qmd).
